In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: AutoML video classification model


## Installation

Install the latest (preview) version of Vertex SDK.


In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import json
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML constants

Next, setup constants unique to AutoML video classification datasets and training:

- Dataset Schemas: Tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas: Tells the Vertex AI Pipelines service the task (e.g., classification) to train the model for.


In [ ]:
# Video Dataset type
VIDEO_SCHEMA = "google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml"
# Video Labeling type
IMPORT_SCHEMA_VIDEO_CLASSIFICATION = "gs://google-cloud-aiplatform/schema/dataset/ioformat/video_classification_io_format_1.0.0.yaml"
# Video Training task
TRAINING_VIDEO_CLASSIFICATION_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_classification_1.0.0.yaml"

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10

*Example output*:
```
gs://automl-video-demo-data/hmdb51/_Rad_Schlag_die_Bank__cartwheel_f_cm_np1_le_med_0.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_ba_bad_8.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_fr_bad_3.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_fr_bad_4.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_fr_bad_5.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Bayer__Meisterin_Teresa_Stadler_cartwheel_f_cm_np1_le_med_0.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Bayer__Meisterin_Teresa_Stadler_cartwheel_f_cm_np1_le_med_2.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Boden_bung_Spoho_Eignungspr_fung_cartwheel_f_cm_np1_ri_med_2.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Bodenturnen_2004_cartwheel_f_cm_np1_le_med_0.avi,cartwheel,0.0,inf
gs://automl-video-demo-data/hmdb51/Bodenturnen_2004_cartwheel_f_cm_np1_le_med_4.avi,cartwheel,0.0,inf
```


## Create a dataset


### [projects.locations.datasets.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/create)


#### Request


In [ ]:
DATA_SCHEMA = VIDEO_SCHEMA

dataset = {
    "display_name": "hmdb_" + TIMESTAMP,
    "metadata_schema_uri": "gs://" + DATA_SCHEMA,
}

print(
    MessageToJson(
        aip.CreateDatasetRequest(parent=PARENT, dataset=dataset).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "hmdb_20210228191029",
    "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml"
  }
}
```


#### Call


In [ ]:
request = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/7952037527982964736",
  "displayName": "hmdb_20210228191029",
  "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/video_1.0.0.yaml",
  "labels": {
    "aiplatform.googleapis.com/dataset_metadata_schema": "VIDEO"
  },
  "metadata": {
    "dataItemSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/dataitem/video_1.0.0.yaml"
  }
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

# Saved for clean up
dataset = {"name": dataset_id}

### [projects.locations.datasets.import](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/import)


#### Request


In [ ]:
LABEL_SCHEMA = IMPORT_SCHEMA_VIDEO_CLASSIFICATION

import_config = {
    "gcs_source": {"uris": [IMPORT_FILE]},
    "import_schema_uri": LABEL_SCHEMA,
}

print(
    MessageToJson(
        aip.ImportDataRequest(
            name=dataset_short_id, import_configs=[import_config]
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "7952037527982964736",
  "importConfigs": [
    {
      "gcsSource": {
        "uris": [
          "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"
        ]
      },
      "importSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/ioformat/video_classification_io_format_1.0.0.yaml"
    }
  ]
}
```


#### Call


In [ ]:
request = clients["dataset"].import_data(
    name=dataset_id, import_configs=[import_config]
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


## Train a model


### [projects.locations.trainingPipelines.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)


#### Request


In [ ]:
TRAINING_SCHEMA = TRAINING_VIDEO_CLASSIFICATION_SCHEMA

task = ParseDict({}, Value())

training_pipeline = {
    "display_name": "hmdb_" + TIMESTAMP,
    "input_data_config": {"dataset_id": dataset_short_id},
    "model_to_upload": {"display_name": "hmdb_" + TIMESTAMP},
    "training_task_definition": TRAINING_SCHEMA,
    "training_task_inputs": task,
}

print(
    MessageToJson(
        aip.CreateTrainingPipelineRequest(
            parent=PARENT, training_pipeline=training_pipeline
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "trainingPipeline": {
    "displayName": "hmdb_20210228191029",
    "inputDataConfig": {
      "datasetId": "7952037527982964736"
    },
    "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_classification_1.0.0.yaml",
    "trainingTaskInputs": {},
    "modelToUpload": {
      "displayName": "hmdb_20210228191029"
    }
  }
}
```


#### Call


In [ ]:
request = clients["pipeline"].create_training_pipeline(
    parent=PARENT, training_pipeline=training_pipeline
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/3361945917925097472",
  "displayName": "hmdb_20210228191029",
  "inputDataConfig": {
    "datasetId": "7952037527982964736"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_classification_1.0.0.yaml",
  "trainingTaskInputs": {},
  "modelToUpload": {
    "displayName": "hmdb_20210228191029"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-28T19:17:39.815377Z",
  "updateTime": "2021-02-28T19:17:39.815377Z"
}
```


In [ ]:
# The full unique ID for the training pipeline
training_pipeline_id = request.name
# The short numeric ID for the training pipeline
training_pipeline_short_id = training_pipeline_id.split("/")[-1]

print(training_pipeline_id)

### [projects.locations.trainingPipelines.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)


#### Call


In [ ]:
request = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/3361945917925097472",
  "displayName": "hmdb_20210228191029",
  "inputDataConfig": {
    "datasetId": "7952037527982964736"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_video_classification_1.0.0.yaml",
  "trainingTaskInputs": {},
  "modelToUpload": {
    "displayName": "hmdb_20210228191029"
  },
  "state": "PIPELINE_STATE_RUNNING",
  "createTime": "2021-02-28T19:17:39.815377Z",
  "startTime": "2021-02-28T19:17:40.089331Z",
  "updateTime": "2021-02-28T19:17:40.089331Z"
}
```


In [ ]:
while True:
    response = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_name = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        model_id = response.model_to_upload.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(20)

print(model_id)

## Evaluate the model


### [projects.locations.models.evaluations.list](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)


#### Call


In [ ]:
request = clients["model"].list_model_evaluations(parent=model_id)

#### Response


In [ ]:
model_evaluations = [json.loads(MessageToJson(mel.__dict__["_pb"])) for mel in request]

print(json.dumps(model_evaluations, indent=2))

# The evaluation slice
evaluation_slice = request.model_evaluations[0].name

*Example output*
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/5031242063400665088/evaluations/6719412425478635520",
    "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    "metrics": {
      "confidenceMetrics": [
        {
          "confidenceThreshold": 0.0891612,
          "precision": 0.2,
          "recall": 1.0,
          "f1Score": 0.33333334
        },
        {
          "recall": 1.0,
          "confidenceThreshold": 0.09073429,
          "precision": 0.20289855,
          "f1Score": 0.33734939
        },
        {
          "recall": 1.0,
          "f1Score": 0.34146342,
          "confidenceThreshold": 0.09176466,
          "precision": 0.20588236
        },
        
        # REMOVED FOR BREVITY
        
        {
          {
            "displayName": "pullup",
            "id": "2856417959264387072"
          },
          {
            "displayName": "golf",
            "id": "5162260968478081024"
          },
          {
            "displayName": "ride_horse",
            "id": "6315182473084928000"
          },
          {
            "displayName": "cartwheel",
            "id": "7468103977691774976"
          }
        ]
      }
    },
    "createTime": "2021-02-28T20:56:43.050002Z",
    "sliceDimensions": [
      "annotationSpec"
    ]
  }
]
```


### [projects.locations.models.evaluations.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/get)


#### Call


In [ ]:
request = clients["model"].get_model_evaluation(name=evaluation_slice)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/5031242063400665088/evaluations/6719412425478635520",
  "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
  "metrics": {
    "confusionMatrix": {
      "rows": [
        [
          14.0,
          0.0,
          0.0,
          0.0,
          0.0
        ],
        [
          0.0,
          14.0,
          0.0,
          0.0,
          0.0
        ],
        [
          0.0,
          0.0,
          14.0,
          0.0,
          0.0
        ],
        [
          0.0,
          0.0,
          0.0,
          14.0,
          0.0
        ],
        [
          0.0,
          0.0,
          0.0,
          0.0,
          14.0
        ]
      ],
      "annotationSpecs": [
        {
          "displayName": "kick_ball",
          "id": "1703496454657540096"
        },
        {
          "displayName": "pullup",
          "id": "2856417959264387072"
        },
        {
          "displayName": "golf",
          "id": "5162260968478081024"
        },
        {
          "displayName": "ride_horse",
          "id": "6315182473084928000"
        },
        {
          "displayName": "cartwheel",
          "id": "7468103977691774976"
        }
      ]
    },
    "confidenceMetrics": [
      {
        "precision": 0.2,
        "recall": 1.0,
        "confidenceThreshold": 0.0891612,
        "f1Score": 0.33333334
      },
      {
        "recall": 1.0,
        "f1Score": 0.33734939,
        "confidenceThreshold": 0.09073429,
        "precision": 0.20289855
      },
      {
        "precision": 0.20588236,
        "f1Score": 0.34146342,
        "confidenceThreshold": 0.09176466,
        "recall": 1.0
      },
      {
        "confidenceThreshold": 0.09279072,
        "f1Score": 0.34739456,
        "precision": 0.2102102,
        "recall": 1.0
      },
      
      # REMOVED FOR BREVITY
      
      {
        "recall": 0.071428575,
        "f1Score": 0.13333334,
        "precision": 1.0,
        "confidenceThreshold": 0.6023364
      },
      {
        "f1Score": 0.055555556,
        "precision": 1.0,
        "confidenceThreshold": 0.6101756,
        "recall": 0.028571429
      },
      {
        "recall": 0.014285714,
        "precision": 1.0,
        "confidenceThreshold": 0.6113689,
        "f1Score": 0.028169014
      }
    ],
    "auPrc": 1.0
  },
  "createTime": "2021-02-28T20:56:43.050002Z",
  "sliceDimensions": [
    "annotationSpec"
  ]
}
```


## Make batch predictions


### Make a batch prediction file


In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

cols = str(test_items[0]).split(",")
test_item_1 = str(cols[0])
test_label_1 = str(cols[1])

print(test_item_1, test_label_1)

cols = str(test_items[1]).split(",")
test_item_2 = str(cols[0])
test_label_2 = str(cols[1])

print(test_item_2, test_label_2)

*Example output*:
```
gs://automl-video-demo-data/hmdb51/_Rad_Schlag_die_Bank__cartwheel_f_cm_np1_le_med_0.avi cartwheel
gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_ba_bad_8.avi cartwheel
```


### Make the batch input file

Let's now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each video. The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the video.
- `mimeType`: The content type. In our example, it is an `avi` file.
- `timeSegmentStart`: The start timestamp in the video to do prediction on. *Note*, the timestamp must be specified as a string and followed by s (second), m (minute) or h (hour).
- `timeSegmentEnd`: The end timestamp in the video to do prediction on.


In [ ]:
import json

import tensorflow as tf

gcs_input_uri = "gs://" + BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {
        "content": test_item_1,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "inf",
    }
    f.write(json.dumps(data) + "\n")
    data = {
        "content": test_item_2,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "inf",
    }
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)

!gsutil cat $gcs_input_uri

*Example output*:
```
gs://migration-ucaip-trainingaip-20210228191029/test.jsonl
{"content": "gs://automl-video-demo-data/hmdb51/_Rad_Schlag_die_Bank__cartwheel_f_cm_np1_le_med_0.avi", "mimeType": "video/avi", "timeSegmentStart": "0.0s", "timeSegmentEnd": "inf"}
{"content": "gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_ba_bad_8.avi", "mimeType": "video/avi", "timeSegmentStart": "0.0s", "timeSegmentEnd": "inf"}
```


### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)


#### Request


In [ ]:
batch_prediction_job = {
    "display_name": "hmdb_" + TIMESTAMP,
    "model": model_id,
    "input_config": {
        "instances_format": "jsonl",
        "gcs_source": {"uris": [gcs_input_uri]},
    },
    "model_parameters": ParseDict(
        {
            "confidenceThreshold": 0.5,
            "maxPredictions": 2,
            "segmentClassification": True,
            "shotClassification": True,
            "oneSecIntervalClassification": True,
        },
        Value(),
    ),
    "output_config": {
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
        },
    },
    "dedicated_resources": {
        "machine_spec": {"machine_type": "n1-standard-2", "accelerator_count": 0},
        "starting_replica_count": 1,
        "max_replica_count": 1,
    },
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT, batch_prediction_job=batch_prediction_job
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "hmdb_20210228191029",
    "model": "projects/116273516712/locations/us-central1/models/5031242063400665088",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210228191029/test.jsonl"
        ]
      }
    },
    "modelParameters": {
      "segmentClassification": true,
      "maxPredictions": 2.0,
      "shotClassification": true,
      "confidenceThreshold": 0.5,
      "oneSecIntervalClassification": true
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210228191029/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```


#### Call


In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT, batch_prediction_job=batch_prediction_job
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/5275975759557558272",
  "displayName": "hmdb_20210228191029",
  "model": "projects/116273516712/locations/us-central1/models/5031242063400665088",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210228191029/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "oneSecIntervalClassification": true,
    "confidenceThreshold": 0.5,
    "maxPredictions": 2.0,
    "shotClassification": true,
    "segmentClassification": true
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210228191029/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-02-28T22:47:21.875565Z",
  "updateTime": "2021-02-28T22:47:21.875565Z"
}
```


In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)


#### Call


In [ ]:
request = clients["job"].get_batch_prediction_job(name=batch_job_id)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/5275975759557558272",
  "displayName": "hmdb_20210228191029",
  "model": "projects/116273516712/locations/us-central1/models/5031242063400665088",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210228191029/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "oneSecIntervalClassification": true,
    "confidenceThreshold": 0.5,
    "shotClassification": true,
    "maxPredictions": 2.0,
    "segmentClassification": true
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210228191029/batch_output/"
    }
  },
  "state": "JOB_STATE_RUNNING",
  "completionStats": {
    "incompleteCount": "2"
  },
  "createTime": "2021-02-28T22:47:21.875565Z",
  "startTime": "2021-02-28T22:47:22.041508Z",
  "updateTime": "2021-02-28T22:47:22.486289Z"
}
```


In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(
            response.output_config.gcs_destination.output_uri_prefix
        )
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)

*Example output*:
```
gs://migration-ucaip-trainingaip-20210228191029/batch_output/prediction-hmdb_20210228191029-2021-02-28T22:47:21.701608Z/predictions_00001.jsonl
gs://migration-ucaip-trainingaip-20210228191029/batch_output/prediction-hmdb_20210228191029-2021-02-28T22:47:21.701608Z/predictions_00002.jsonl
{"instance":{"content":"gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_ba_bad_8.avi","mimeType":"video/avi","timeSegmentStart":"0.0s","timeSegmentEnd":"inf"},"prediction":[]}
{"instance":{"content":"gs://automl-video-demo-data/hmdb51/_Rad_Schlag_die_Bank__cartwheel_f_cm_np1_le_med_0.avi","mimeType":"video/avi","timeSegmentStart":"0.0s","timeSegmentEnd":"inf"},"prediction":[{"id":"7468103977691774976","displayName":"cartwheel","type":"shot-classification","timeSegmentStart":"0.066666s","timeSegmentEnd":"0.226666s","confidence":0.5290586},{"id":"7468103977691774976","displayName":"cartwheel","type":"one-sec-interval-classification","timeSegmentStart":"1.346666s","timeSegmentEnd":"1.346666s","confidence":0.5290586},{"id":"7468103977691774976","displayName":"cartwheel","type":"segment-classification","timeSegmentStart":"0s","timeSegmentEnd":"2.766667s","confidence":0.52444863},{"id":"7468103977691774976","displayName":"cartwheel","type":"shot-classification","timeSegmentStart":"0.266666s","timeSegmentEnd":"2.226666s","confidence":0.51983875},{"id":"7468103977691774976","displayName":"cartwheel","type":"one-sec-interval-classification","timeSegmentStart":"1.586666s","timeSegmentEnd":"1.586666s","confidence":0.51983875}]}
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_dataset = True
delete_model = True
delete_pipeline = True
delete_batchjob = True
delete_bucket = True

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    if delete_model:
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex AI fully qualified identifier for the training pipeline
try:
    if delete_pipeline:
        clients["pipeline"].delete_training_pipeline(name=training_pipeline_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex AI fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME